In [1]:
import os
import itertools
import datetime as dt
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Noto Sans CJK JP'
import seaborn as sns
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils.helper import fix_seed, seed_worker
fix_seed()
from dataset import STDataset
from trainer import Trainer
from logger import Logger
import config

In [2]:
import transformer

In [2]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=3)

## dfを前処理 → pickle形式のデータセットを作成

In [2]:
# !python3.9 df_preprocess.py ./datasets_1h/all.pkl -s 2021/4/2 -e 2022/6/1
# !python3.9 df_preprocess.py ./datasets_1h/mini.pkl -s 2021/4/2 -e 2021/6/1

In [3]:
# train
# !python3.9 df_preprocess.py ./datasets_1h/all.pkl -s 2021/4/2 -e 2022/3/1
# test
# !python3.9 df_preprocess.py ./datasets_1h/mini.pkl -s 2022/3/1 -e 2022/6/1

In [4]:
# mini
df_test = pd.read_pickle('./datasets_1h/kannetsu_210402-210531.pkl')
# whole
df_all = pd.read_pickle('./datasets_1h/kannetsu_210402-220531.pkl')

## Spatial Temporal Matrixに整形 & 標準化
- 区間数 x 時系列数 の行列
- 実際は 区間数 x 時系列数 x 特徴量数 のテンソル

- 標準化を行う
- (検索数, 台数は上り・下り別でもやってみる)

In [ ]:
!python3.9 transformer.py -s ./train.pkl ./features_train_norm.pkl ./labels_train.pkl --scaler_path ./datasets/training_scaler.pkl
!python3.9 transformer.py -s ./test.pkl ./features_test_norm.pkl ./labels_test.pkl --scaler_path ./datasets/training_scaler.pkl

convert matrix ((72576, 42)) to tensor ...


In [36]:
# train
# !python3.9 transformer.py -s ./datasets_1h/df_train.pkl ./datasets_1h/features_train_norm.pkl ./datasets_1h/labels_train.pkl --scaler_path ./datasets/training_scaler.pkl
# test
# !python3.9 transformer.py -s ./datasets_1h/df_test.pkl ./datasets_1h/features_test_norm.pkl ./datasets_1h/labels_test.pkl --scaler_path ./datasets/training_scaler.pkl

In [3]:
df_test_tr = pd.read_pickle('./datasets_1h/kannetsu_210402-210519.pkl')
df_test_va = pd.read_pickle('./datasets_1h/kannetsu_210520-210531.pkl')

df_all_tr = pd.read_pickle('./datasets_1h/kannetsu_210402-220228.pkl')
df_all_va = pd.read_pickle('./datasets_1h/kannetsu_220301-220531.pkl')

dt_table = pd.read_pickle(f'{config.TABLES_DIR}/datetime_table.pkl')
sec_table = pd.read_pickle(f'{config.TABLES_DIR}/section_table.pkl')

In [4]:
# 特徴量の元になる列
# time_col = ['month', 'hour', 'dayofweek', 'is_holiday']
# section_col = ['direction', 'lane_count', 'KP']
time_col = ['datetime_id']
section_col = ['section_id']
search_col = ['search_1h', 'search_unspec_1d']
traffic_col = ['allCars']

feature_col = time_col + section_col + search_col + traffic_col
# feature_col = time_col + section_col + traffic_col
# feature_col = search_col + traffic_col

# 予測対象
target_col = 'allCars'

In [18]:
X_tr, y_tr = transformer.matrix2tensor(df_all_tr, sec_table, feature_col, config.TARGET_COL)
X_va, y_va = transformer.matrix2tensor(df_all_va, sec_table, feature_col, config.TARGET_COL)
print(X_tr.shape, X_va.shape, y_tr.shape, y_va.shape)

torch.Size([5, 7992, 63]) torch.Size([5, 2208, 63]) torch.Size([1, 7992, 63]) torch.Size([1, 2208, 63])


In [30]:
dt_table = pd.read_pickle(f'{config.TABLES_DIR}/datetime_table.pkl')
sec_table = pd.read_pickle(f'{config.TABLES_DIR}/section_table.pkl')

# X_tr = torch.load('./datasets_1h/features_train.pkl')
# X_va = torch.load('./datasets_1h/features_test.pkl')
# y_tr = torch.load('./datasets_1h/labels_train.pkl')
# y_va = torch.load('./datasets_1h/labels_test.pkl')

X_tr = torch.load(f'datasets_1h/mini_features_train.pkl')
X_va = torch.load(f'datasets_1h/mini_features_test.pkl')
y_tr = torch.load(f'datasets_1h/mini_labels_train.pkl')
y_va = torch.load(f'datasets_1h/mini_labels_test.pkl')

print(X_tr.shape, X_va.shape)
print(y_tr.shape, y_va.shape)

torch.Size([5, 1152, 63]) torch.Size([5, 288, 63])
torch.Size([1, 1152, 63]) torch.Size([1, 288, 63])


In [31]:
X_tr = torch.load('./t_train.pkl')
X_va = torch.load('./t_test.pkl')
y_tr = torch.load('./l_train.pkl')
y_va = torch.load('./l_test.pkl')

print(X_tr.shape, X_va.shape)
print(y_tr.shape, y_va.shape)

torch.Size([5, 1152, 63]) torch.Size([5, 288, 63])
torch.Size([1, 1152, 63]) torch.Size([1, 288, 63])


In [ ]:
# ID列は飛ばして標準化
skip_features = [0, 1]
scaler = STMatrixStandardScaler(skip_features=skip_features)

scaler.fit(X_tr)
X_tr_norm = scaler.transform(X_tr)

scaler.fit(X_va)
X_va_norm = scaler.transform(X_va)

# torch.save(X_tr_norm, f'datasets_1h/features_train_norm.pkl')
# torch.save(X_va_norm, f'datasets_1h/features_test_norm.pkl')

## データセットの定義

In [3]:
dt_table = pd.read_pickle(f'{config.TABLES_DIR}/datetime_table.pkl')
sec_table = pd.read_pickle(f'{config.TABLES_DIR}/section_table.pkl')

X_tr = torch.load(f'{config.DATASET_DIR}_1h/features_train_norm.pkl')
X_va = torch.load(f'{config.DATASET_DIR}_1h/features_test_norm.pkl')
y_tr = torch.load(f'{config.DATASET_DIR}_1h/labels_train.pkl')
y_va = torch.load(f'{config.DATASET_DIR}_1h/labels_test.pkl')

# X_tr = torch.load(f'datasets_1h/mini_features_train_norm.pkl')
# X_va = torch.load(f'datasets_1h/mini_features_test_norm.pkl')
# y_tr = torch.load(f'datasets_1h/mini_labels_train.pkl')
# y_va = torch.load(f'datasets_1h/mini_labels_test.pkl')

print(dt_table.shape, sec_table.shape)
print(X_tr.shape, X_va.shape)
print(y_tr.shape, y_va.shape)

(288, 1) (63, 4)
torch.Size([5, 7992, 63]) torch.Size([5, 2208, 63])
torch.Size([1, 7992, 63]) torch.Size([1, 2208, 63])


In [4]:
def zeroing_feature(tensor, feature_nums):
    new = tensor.clone()
    for f in feature_nums:
        new[f] = 0
    return new

In [27]:
X_tr_wo_search = zeroing_feature(X_tr, [2, 3])
X_tr_wo_search_spec = zeroing_feature(X_tr, [2])
X_tr_wo_search_unspec = zeroing_feature(X_tr, [3])
X_tr_wo_context = zeroing_feature(X_tr, [0, 1])
X_tr_wo_both = zeroing_feature(X_tr, [0, 1, 2, 3])

X_va_wo_search = zeroing_feature(X_va, [2, 3])
X_va_wo_search_spec = zeroing_feature(X_va, [2])
X_va_wo_search_unspec = zeroing_feature(X_va, [3])
X_va_wo_context = zeroing_feature(X_va, [0, 1])
X_va_wo_both = zeroing_feature(X_va, [0, 1, 2, 3])

In [4]:
time_step = 7 * 24
space_window = (-2, 2)

dataset_train = STDataset(X_tr, y_tr, 
                          time_step=time_step, 
                          space_window=space_window)
dataset_valid = STDataset(X_va, y_va, 
                          time_step=time_step, 
                          space_window=space_window)

## Networkの定義

In [5]:
TRAFFIC_CONV = 64
TRAFFIC_HIDDEN = 128
TRAFFIC_LSTM_LAYERS = 4
TRAFFIC_KERNEL = (7,5)

SEARCH_CONV = 64
SEARCH_HIDDEN = 128
SEARCH_LSTM_LAYERS = 4
SEARCH_KERNEL = (7,5)

UNSPEC_SEARCH_HIDDEN = 64
UNSPEC_SEARCH_KERNEL = 5

DATETIME_EMB = 32
ROAD_EMB = 16

FC_EMB = 32

PREDICTION_HORIZON = 24

### T-ED

In [6]:
class LSTMEncoder(nn.Module):
    def __init__(self, lstm_dropout, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(1, TRAFFIC_HIDDEN, TRAFFIC_LSTM_LAYERS, 
                            bidirectional=bidirectional, dropout=lstm_dropout, batch_first=True)
    
    def forward(self, x):
        N, T, S = x.shape
        x = x[..., [S // 2]]
        # N x T x 1 -> N x T x H, (L x N x H, L x N x H)
        outs, (h, c) = self.lstm(x)
        
        if self.bidirectional:
            # 2*L_t x N x H_t -> L_t x N x 2*H_t
            L2, N, H_t = h.shape
            h = h.transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
            c = h.transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
        
        return outs, (h, c)
    

class LSTMDecoder(nn.Module):
    def __init__(self, lstm_dropout, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        
        self.hid_dim = 2 * TRAFFIC_HIDDEN if bidirectional else TRAFFIC_HIDDEN
        
        self.bnorm = nn.BatchNorm1d(1)
        self.lstm = nn.LSTM(1, self.hid_dim, TRAFFIC_LSTM_LAYERS, 
                            dropout=lstm_dropout, batch_first=True)

        self.fc1 = nn.Linear(self.hid_dim, FC_EMB)
        self.fc2 = nn.Linear(FC_EMB, 1)

    def forward(self, x, state):
        N, _, P = x.shape
        # normalize
        x = self.bnorm(x)
        
        # N x C=1 x P -> N x P x C=1
        x = x.permute(0, 2, 1)
        # N x P x C, (L x N x H_t, L x N x H_t) -> N x P x H_t, (L x N x H_t, L x N x H_t)
        outs, state = self.lstm(x, state)
        outs = F.relu(self.fc1(outs))
        outs = self.fc2(outs)

        return outs, state
    
    def generate(self, trf_enc, start_value=-1.0):
        with torch.no_grad():
            # N x 1 x 1
            N = trf_enc[0].shape[1]
            out = torch.tensor(start_value).repeat(N).unsqueeze(-1).unsqueeze(-1)
            out = out.to(trf_enc[0].device)
            
            state = trf_enc
            
            generated = []

            for i in range(24):
                out, state = self.forward(out, state)
                generated.append(out)
        
        # N x P x 1
        generated = torch.cat(generated, dim=1)
        # N x P x 1 -> N x P
        generated = generated[..., 0]
        return generated
    

class LSTMOnlyEncoderDecoder(nn.Module):
    def __init__(self, lstm_dropout=0, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        
        self.encoder = LSTMEncoder(lstm_dropout=lstm_dropout, bidirectional=bidirectional)
        self.decoder = LSTMDecoder(lstm_dropout=lstm_dropout, bidirectional=bidirectional)
        
    def forward(self, features, decoder_xs):
        dt, rd, sr, un_sr, trf = features
        
        outs_trf, state_trf = self.encoder(trf)
        outs, _ = self.decoder(decoder_xs, state_trf)
        outs = outs[..., 0]
        return outs
    
    def generate(self, features, start_value=-1.0):
        dt, rd, sr, un_sr, trf = features
        
        outs_trf, state_trf = self.encoder(trf)    
        generated = self.decoder.generate(state_trf, start_value)
        return generated

### ST-ED, ST-ED-S

In [7]:
class CNNLSTMEncoder(nn.Module):
    def __init__(self, lstm_dropout, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.bidirectional = bidirectional
        
        self.conv = nn.Conv2d(1, TRAFFIC_CONV, TRAFFIC_KERNEL, 
                              padding=(TRAFFIC_KERNEL[0]//2, 0), padding_mode='replicate')
        self.lstm = nn.LSTM(TRAFFIC_CONV, TRAFFIC_HIDDEN, TRAFFIC_LSTM_LAYERS,
                            dropout=lstm_dropout, bidirectional=bidirectional, batch_first=True)
    
    def forward(self, x):
        N, T, S = x.shape
        
        out = F.relu(self.conv(x.unsqueeze(1)))
        # N x C x T -> N x T x C
        out = out[..., 0].permute(0, 2, 1)
        # N x T x C -> N x T x H, (L x N x H, L x N x H)
        outs, (h, c) = self.lstm(out)
        
        if self.bidirectional:
            # 2*L_t x N x H_t -> L_t x N x 2*H_t
            L2, N, H_t = h.shape
            h = h.transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
            c = h.transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
        
        return outs, (h, c)
    

class CNNLSTMEncoderDecoder(LSTMOnlyEncoderDecoder):
    def __init__(self, lstm_dropout=0, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        
        self.encoder = CNNLSTMEncoder(lstm_dropout=lstm_dropout, bidirectional=bidirectional)
        self.decoder = LSTMDecoder(lstm_dropout=lstm_dropout, bidirectional=bidirectional)

### TE-ED, TE-ED-S

In [8]:
class LSTMEmbeddingDecoder(nn.Module):
    def __init__(self, lstm_dropout, embedding_dropout=0.4, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.embedding_dropout = embedding_dropout
        
        self.hid_dim = 2 * TRAFFIC_HIDDEN if bidirectional else TRAFFIC_HIDDEN
        
        self.datetime_embedding = CategoricalEmbedding(config.DT_TABLE_SIZE, DATETIME_EMB)
        self.road_embedding = CategoricalEmbedding(config.SEC_TABLE_SIZE, ROAD_EMB)
        self.emb_dropout = nn.Dropout(p=embedding_dropout)
        
        self.bnorm = nn.BatchNorm1d(1)
        self.lstm = nn.LSTM(1, self.hid_dim, TRAFFIC_LSTM_LAYERS, 
                            dropout=lstm_dropout, batch_first=True)
        self.fc1 = nn.Linear(self.hid_dim + DATETIME_EMB + ROAD_EMB, FC_EMB)
        self.fc2 = nn.Linear(FC_EMB, 1)

    def forward(self, x, state, dt, rd):
        N, _, P = x.shape
        # normalize
        x = self.bnorm(x)
        
        # N x C=1 x P -> N x P x C=1
        x = x.permute(0, 2, 1)
        # N x P x C, (L x N x H_t, L x N x H_t) -> N x P x H_t, (L x N x H_t, L x N x H_t)
        outs, state = self.lstm(x, state)
        
        # N x P -> N x P x H_d
        dt_emb = self.datetime_embedding(dt)
        dt_emb = self.emb_dropout(dt_emb)
        # N x 1 -> N x 1 x H_r
        rd_emb = self.road_embedding(rd)
        rd_emb = self.emb_dropout(rd_emb)
        
        outs = torch.cat([
            dt_emb, 
            rd_emb.repeat(1, P, 1), 
            outs
        ], dim=-1)
        
        outs = F.relu(self.fc1(outs))
        outs = self.fc2(outs)

        return outs, state
    
    def generate(self, trf_enc, dt, rd, start_value=-1.0):
        with torch.no_grad():
            # N x 1 x 1
            N = trf_enc[0].shape[1]
            out = torch.tensor(start_value).repeat(N).unsqueeze(-1).unsqueeze(-1)
            out = out.to(trf_enc[0].device)
            
            state = trf_enc
            
            generated = []

            for i in range(24):
                out, state = self.forward(out, state, dt[:, [i]], rd)
                generated.append(out)
        
        # N x P x 1
        generated = torch.cat(generated, dim=1)
        # N x P x 1 -> N x P
        generated = generated[..., 0]
        return generated
    

class LSTMEmbeddingEncoderDecoder(nn.Module):
    def __init__(self, lstm_dropout=0, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        
        self.encoder = LSTMEncoder(lstm_dropout=lstm_dropout, bidirectional=bidirectional)
        self.decoder = LSTMEmbeddingDecoder(lstm_dropout=lstm_dropout, bidirectional=bidirectional)
        
    def forward(self, features, decoder_xs):
        dt, rd, sr, un_sr, trf = features
        
        outs_trf, state_trf = self.encoder(trf)
        outs, _ = self.decoder(decoder_xs, state_trf, dt, rd)
        outs = outs[..., 0]
        return outs
    
    def generate(self, features, start_value=-1.0):
        dt, rd, sr, un_sr, trf = features
        
        outs_trf, state_trf = self.encoder(trf)    
        generated = self.decoder.generate(state_trf, dt, rd, start_value)
        return generated

In [9]:
class T_TrafficSearchEncoder(nn.Module):
    def __init__(self, conv_dim, kernel, lstm_dim, lstm_layers, lstm_dropout, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.bidirectional = bidirectional
        
        self.lstm = nn.LSTM(1, lstm_dim, lstm_layers, 
                            dropout=lstm_dropout, bidirectional=bidirectional, batch_first=True)
        
    def forward(self, x):
        N, T, S = x.shape
        
        x = x[..., [S // 2]]
        # N x T x C -> N x T x H, (L x N x H, L x N x H)
        outs, (h, c) = self.lstm(x)
        
        return outs, (h, c)

    
class T_SearchUnspecEncoder(nn.Module):
    def __init__(self, p_dropout):
        super().__init__()
        
        self.p_dropout = p_dropout
        self.dropout = nn.Dropout(p=p_dropout)
    
    def forward(self, x):
        N, T, S = x.shape
        
        x = x[..., [S // 2]]
        out = self.dropout(x)
        
        return out

    
class T_Encoder(nn.Module):
    def __init__(
        self, lstm_dropout, unspec_search_dropout=0.4, include_search=True, bidirectional=True
    ):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.unspec_search_dropout = unspec_search_dropout
        self.include_search = include_search
        self.bidirectional = bidirectional
        
        self.traffic_encoder = T_TrafficSearchEncoder(
            TRAFFIC_CONV, TRAFFIC_KERNEL, TRAFFIC_HIDDEN, TRAFFIC_LSTM_LAYERS,
            lstm_dropout=lstm_dropout, bidirectional=bidirectional
        )
        
        if include_search:
            self.search_encoder = T_TrafficSearchEncoder(
                SEARCH_CONV, SEARCH_KERNEL, SEARCH_HIDDEN, SEARCH_LSTM_LAYERS,
                lstm_dropout=lstm_dropout, bidirectional=bidirectional
            )
            self.unspec_search_encoder = T_SearchUnspecEncoder(p_dropout=unspec_search_dropout)
        
    def forward(self, x_trf, x_sr, x_un_sr):
        # N x T x S -> N x T x H_t, (bi*L_t x N x H_t, bi*L_t x N x H_t)
        outs_trf, state_trf = self.traffic_encoder(x_trf)
        if self.bidirectional:
            # 2*L_t x N x H_t -> L_t x N x 2*H_t
            L2, N, H_t = state_trf[0].shape
            h = state_trf[0].transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
            c = state_trf[1].transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
            state_trf = (h, c)
        
        if self.include_search:
            # N x P x S -> N x P x H_s, (L_s x N x H_s, L_s x N x H_s) 
            outs_sr, state_sr = self.search_encoder(x_sr)
            # N x 1 x S -> N x 1 x H_u
            out_un_sr = self.unspec_search_encoder(x_un_sr)
            return (outs_trf, state_trf), outs_sr, out_un_sr
        return outs_trf, state_trf

In [10]:
class T_EncoderDecoder(nn.Module):
    def __init__(self, lstm_dropout=0, include_search=True, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.include_search = include_search
        
        self.encoder = T_Encoder(
            lstm_dropout=lstm_dropout, include_search=include_search, bidirectional=bidirectional
        )
        self.decoder = Decoder(
            lstm_dropout=lstm_dropout, include_search=include_search, bidirectional=bidirectional
        )
        
    def forward(self, features, decoder_xs):
        dt, rd, sr, un_sr, trf = features
        
        if self.include_search:
            (outs_trf, state_trf), outs_sr, outs_un_sr = self.encoder(trf, sr, un_sr)
        else:
            outs_trf, state_trf = self.encoder(trf, sr, un_sr)
            outs_sr, outs_un_sr = None, None
            
        outs = self.decoder(decoder_xs, state_trf, outs_sr, outs_un_sr, dt, rd)

        return outs
    
    def generate(self, features, start_value=-1.0):
        dt, rd, sr, un_sr, trf = features
        
        if self.include_search:
            (outs_trf, state_trf), outs_sr, outs_un_sr = self.encoder(trf, sr, un_sr)
        else:
            outs_trf, state_trf = self.encoder(trf, sr, un_sr)
            outs_sr, outs_un_sr = None, None
        
        generated = self.decoder.generate(state_trf, outs_sr, outs_un_sr, dt, rd, start_value)
        return generated

### STE-ED, STE-ED-S

In [11]:
class TrafficSearchEncoder(nn.Module):
    def __init__(self, conv_dim, kernel, lstm_dim, lstm_layers, lstm_dropout, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.bidirectional = bidirectional
        
        self.conv = nn.Conv2d(1, conv_dim, kernel, 
                              padding=(kernel[0]//2, 0), padding_mode='replicate')
        self.lstm = nn.LSTM(conv_dim, lstm_dim, lstm_layers, 
                            dropout=lstm_dropout, bidirectional=bidirectional, batch_first=True)
        
    def forward(self, x):
        N, T, S = x.shape
        
        out = F.relu(self.conv(x.unsqueeze(1)))
        # N x C x T -> N x T x C
        out = out[..., 0].permute(0, 2, 1)
        # N x T x C -> N x T x H, (L x N x H, L x N x H)
        outs, (h, c) = self.lstm(out)
        
        return outs, (h, c)

    
class SearchUnspecEncoder(nn.Module):
    def __init__(self, p_dropout):
        super().__init__()
        
        self.p_dropout = p_dropout
        
        self.conv = nn.Conv1d(1, UNSPEC_SEARCH_HIDDEN, UNSPEC_SEARCH_KERNEL, padding_mode='replicate')
        # self.dropout = nn.Dropout(p=0)
        self.dropout = nn.Dropout(p=p_dropout)
    
    def forward(self, x):
        N, T, S = x.shape
        out = F.relu(self.conv(x))
        # N x C x T -> N x T x C
        out = out.permute(0, 2, 1)
        out = self.dropout(out)
        
        return out

    
class Encoder(nn.Module):
    def __init__(
        self, lstm_dropout, unspec_search_dropout=0.4, include_search=True, bidirectional=True
    ):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.unspec_search_dropout = unspec_search_dropout
        self.include_search = include_search
        self.bidirectional = bidirectional
        
        self.traffic_encoder = TrafficSearchEncoder(
            TRAFFIC_CONV, TRAFFIC_KERNEL, TRAFFIC_HIDDEN, TRAFFIC_LSTM_LAYERS,
            lstm_dropout=lstm_dropout, bidirectional=bidirectional
        )
        
        if include_search:
            self.search_encoder = TrafficSearchEncoder(
                SEARCH_CONV, SEARCH_KERNEL, SEARCH_HIDDEN, SEARCH_LSTM_LAYERS,
                lstm_dropout=lstm_dropout, bidirectional=bidirectional
            )
            self.unspec_search_encoder = SearchUnspecEncoder(p_dropout=unspec_search_dropout)
        
    def forward(self, x_trf, x_sr, x_un_sr):
        # N x T x S -> N x T x H_t, (bi*L_t x N x H_t, bi*L_t x N x H_t)
        outs_trf, state_trf = self.traffic_encoder(x_trf)
        if self.bidirectional:
            # 2*L_t x N x H_t -> L_t x N x 2*H_t
            L2, N, H_t = state_trf[0].shape
            h = state_trf[0].transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
            c = state_trf[1].transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
            state_trf = (h, c)
        
        if self.include_search:
            # N x P x S -> N x P x H_s, (L_s x N x H_s, L_s x N x H_s) 
            outs_sr, state_sr = self.search_encoder(x_sr)
            # N x 1 x S -> N x 1 x H_u
            out_un_sr = self.unspec_search_encoder(x_un_sr)
            return (outs_trf, state_trf), outs_sr, out_un_sr
        return outs_trf, state_trf

In [12]:
class CategoricalEmbedding(nn.Module):
    def __init__(self, category_size, emb_size):
        super().__init__()

        self.category_size = category_size
        self.emb_size = emb_size
        self.embedding = nn.Embedding(category_size, emb_size)

    def forward(self, x):
        out = self.embedding(x.to(dtype=torch.int64))
        return out

class TrafficDecoder(nn.Module):
    def __init__(self, lstm_dropout, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        
        self.hid_dim = 2 * TRAFFIC_HIDDEN if bidirectional else TRAFFIC_HIDDEN
        
        self.bnorm = nn.BatchNorm1d(1)
        self.lstm = nn.LSTM(1, self.hid_dim, TRAFFIC_LSTM_LAYERS, 
                            dropout=lstm_dropout, batch_first=True)

    def forward(self, x, state):
        N, _, P = x.shape
        # normalize
        x = self.bnorm(x)
        
        # N x C=1 x P -> N x P x C=1
        x = x.permute(0, 2, 1)
        # N x P x C, (L x N x H_t, L x N x H_t) -> N x P x H_t, (L x N x H_t, L x N x H_t)
        outs, state = self.lstm(x, state)
        return outs, state

class AffineDecoder(nn.Module):
    def __init__(self, embedding_dropout, include_search=True, bidirectional=True):
        super().__init__()
        
        self.embedding_dropout = embedding_dropout
        self.include_search = include_search
        self.bidirectional = bidirectional
        
        if include_search:
            if bidirectional:
                self.n_dim = 2*TRAFFIC_HIDDEN + 2*SEARCH_HIDDEN + UNSPEC_SEARCH_HIDDEN + DATETIME_EMB + ROAD_EMB
            else:
                self.n_dim = TRAFFIC_HIDDEN + SEARCH_HIDDEN + UNSPEC_SEARCH_HIDDEN + DATETIME_EMB + ROAD_EMB
        else:
            if bidirectional:
                self.n_dim = 2*TRAFFIC_HIDDEN + DATETIME_EMB + ROAD_EMB
            else:
                self.n_dim = TRAFFIC_HIDDEN + DATETIME_EMB + ROAD_EMB
        
        self.datetime_embedding = CategoricalEmbedding(config.DT_TABLE_SIZE, DATETIME_EMB)
        self.road_embedding = CategoricalEmbedding(config.SEC_TABLE_SIZE, ROAD_EMB)
        self.emb_dropout = nn.Dropout(p=embedding_dropout)
        
        self.fc1 = nn.Linear(self.n_dim, FC_EMB)
        self.fc2 = nn.Linear(FC_EMB, 1)
        
    def forward(self, trf_dec, sr_enc, un_sr_enc, dt, rd):
        # traffic_dec: N x P x H_t
        N, P, H_t = trf_dec.shape
        
        # N x P -> N x P x H_d
        dt_emb = self.datetime_embedding(dt)
        dt_emb = self.emb_dropout(dt_emb)
        # N x 1 -> N x 1 x H_r
        rd_emb = self.road_embedding(rd)
        rd_emb = self.emb_dropout(rd_emb)
        
        if self.include_search:
            outs = torch.cat([
                dt_emb, 
                rd_emb.repeat(1, P, 1), 
                sr_enc, 
                un_sr_enc.repeat(1, P, 1),
                trf_dec
            ], dim=-1)
        else:
            outs = torch.cat([
                dt_emb, 
                rd_emb.repeat(1, P, 1), 
                trf_dec
            ], dim=-1)
        # N x P x SUM -> N x P x H_fc
        outs = F.relu(self.fc1(outs))
        # N x P x H_fc -> N x P x 1
        outs = self.fc2(outs)
        
        return outs

class Decoder(nn.Module):
    def __init__(self, lstm_dropout, embedding_dropout=0.4, include_search=True, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.embedding_dropout = embedding_dropout
        self.include_search = include_search
        
        self.traffic_decoder = TrafficDecoder(lstm_dropout=lstm_dropout, bidirectional=bidirectional)
        self.affine_decoder = AffineDecoder(
            embedding_dropout=embedding_dropout, include_search=include_search, 
            bidirectional=bidirectional
        )
    
    def forward(self, x, trf_enc, sr_enc, un_sr_enc, dt, rd):
        # N x 1 x P, (L x N x H_t, L x N x H_t) -> N x P x H_t, (L x N x H_t, L x N x H_t)
        outs_trf, state_trf = self.traffic_decoder(x, trf_enc)
        outs = self.affine_decoder(outs_trf, sr_enc, un_sr_enc, dt, rd)
        # N x P x 1 -> N x P
        outs = outs[..., 0]
        return outs
    
    def generate(self, trf_enc, sr_enc, un_sr_enc, dt, rd, start_value=-1.0):
        with torch.no_grad():
            # N x 1 x 1
            N = trf_enc[0].shape[1]
            out = torch.tensor(start_value).repeat(N).unsqueeze(-1).unsqueeze(-1)
            out = out.to(trf_enc[0].device)
            
            state = trf_enc
            
            generated = []

            for i in range(PREDICTION_HORIZON):
                out, state = self.traffic_decoder(out, state)
                if self.include_search:
                    out = self.affine_decoder(out, sr_enc[:, [i]], un_sr_enc, dt[:, [i]], rd)
                else:
                    out = self.affine_decoder(out, None, None, dt[:, [i]], rd)

                generated.append(out)
        
        # N x P x 1
        generated = torch.cat(generated, dim=1)
        # N x P x 1 -> N x P
        generated = generated[..., 0]
        return generated

In [13]:
class EncoderDecoder(nn.Module):
    def __init__(self, lstm_dropout=0, include_search=True, bidirectional=True):
        super().__init__()
        
        self.lstm_dropout = lstm_dropout
        self.include_search = include_search
        
        self.encoder = Encoder(
            lstm_dropout=lstm_dropout, include_search=include_search, bidirectional=bidirectional
        )
        self.decoder = Decoder(
            lstm_dropout=lstm_dropout, include_search=include_search, bidirectional=bidirectional
        )
        
    def forward(self, features, decoder_xs):
        dt, rd, sr, un_sr, trf = features
        
        if self.include_search:
            (outs_trf, state_trf), outs_sr, outs_un_sr = self.encoder(trf, sr, un_sr)
        else:
            outs_trf, state_trf = self.encoder(trf, sr, un_sr)
            outs_sr, outs_un_sr = None, None
            
        outs = self.decoder(decoder_xs, state_trf, outs_sr, outs_un_sr, dt, rd)

        return outs
    
    def generate(self, features, start_value=-1.0):
        dt, rd, sr, un_sr, trf = features
        
        if self.include_search:
            (outs_trf, state_trf), outs_sr, outs_un_sr = self.encoder(trf, sr, un_sr)
        else:
            outs_trf, state_trf = self.encoder(trf, sr, un_sr)
            outs_sr, outs_un_sr = None, None
        
        generated = self.decoder.generate(state_trf, outs_sr, outs_un_sr, dt, rd, start_value)
        return generated

In [15]:
(dt, rd, sr, un_sr, trf), labels = dataset_train[:16]
print(trf.shape, labels.shape)

torch.Size([16, 168, 5]) torch.Size([16, 1, 24])


In [26]:
(outs_trf, state_trf), outs_sr, outs_un_sr = Encoder(lstm_dropout=0, include_search=True, bidirectional=True)(trf, sr, un_sr)
print(outs_trf.shape, state_trf[0].shape, outs_sr.shape, outs_un_sr.shape)

# (outs_trf, state_trf) = WithoutEncoder(bidirectional=False)(trf)
# print(outs_trf.shape, state_trf[0].shape)

torch.Size([16, 168, 256]) torch.Size([4, 16, 256]) torch.Size([16, 24, 256]) torch.Size([16, 1, 64])


In [ ]:
predicted = Decoder(lstm_dropout=0.4, include_search=True, bidirectional=True).generate(state_trf, outs_sr, outs_un_sr, dt, rd)

# predicted = WithoutDecoder(bidirectional=False).generate(state_trf, dt, rd)

In [32]:
predicted = EncoderDecoder(include_search=False, bidirectional=False).generate((dt, rd, sr, un_sr, trf))

In [33]:
features, labels = dataset_train[:16]

In [34]:
decoder_xs = torch.full_like(labels, -1)
decoder_xs[..., 1:] = labels[..., :-1]

In [35]:
out = EncoderDecoder(include_search=False)((dt, rd, sr, un_sr, trf), decoder_xs)
out.shape

torch.Size([16, 24])

## 学習

In [14]:
def get_loader(dataset, batch_size=config.BATCH_SIZE, shuffle=False):
    g = torch.Generator()
    g.manual_seed(config.RANDOM_SEED)

    dataloader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=shuffle, worker_init_fn=seed_worker, generator=g)
    return dataloader


def train(n_epochs, dataset_train, dataset_valid, model, 
          model_name=None, log_path=None, random_state=config.RANDOM_SEED):
    fix_seed(random_state)
    
    train_loader = get_loader(dataset_train, shuffle=True)
    val_loader = get_loader(dataset_valid, shuffle=False)
    
    # model
    model = model.to(device)
    optimizer = optim.Adam(model.parameters())
    loss_fn = nn.L1Loss()

    logger = Logger(fname=log_path)
    trainer = Trainer(model, optimizer, loss_fn, device=device, logger=logger, model_name=model_name)
    
    train_losses, val_losses = \
        trainer.fit(train_loader, val_loader, n_epochs, log_steps=1)

    return trainer


def test_reproducibility(model_class, lstm_dropout, n_exp=2, n_epochs=2):
    for i in range(n_exp):
        print(':'*25, f'Exp.{i+1}', ':'*25)
        
        model = model_class(lstm_dropout=lstm_dropout)
        trainer = train(n_epochs, model, model_name=None, log_path=None)
        
        print(':'*25, f'best loss: {min(trainer.val_losses):.3f}', ':'*25)
    print()

### 再現性確認

In [17]:
lstm_layers_list = range(1, 4+1)

for lstm_layers in lstm_layers_list:
    # model_name = f'EncDec_LSTM{lstm_layers}'
    # log_path = f'./logs/{model_name}.log'
    model_name, log_path = None, None
    
    print('='*40, f'LSTM layers: {lstm_layers}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    test_reproducibility(EncoderDecoder, lstm_dropout=0, n_exp=3)

======================================== LSTM layers: 1 ========================================
::::::::::::::::::::::::: Exp.1 :::::::::::::::::::::::::
2022-09-21 09:02:56.205079 | Epoch: 1 | Train Loss: 864.096, Train Time: 3.76 [sec] | Valid Loss: 809.587, Valid Time: 0.58 [sec]
2022-09-21 09:02:58.565811 | Epoch: 2 | Train Loss: 687.757, Train Time: 1.70 [sec] | Valid Loss: 686.086, Valid Time: 0.66 [sec]
::::::::::::::::::::::::: best loss: 686.086 :::::::::::::::::::::::::
::::::::::::::::::::::::: Exp.2 :::::::::::::::::::::::::
2022-09-21 09:03:00.854981 | Epoch: 1 | Train Loss: 864.096, Train Time: 1.70 [sec] | Valid Loss: 809.587, Valid Time: 0.58 [sec]
2022-09-21 09:03:03.214122 | Epoch: 2 | Train Loss: 687.757, Train Time: 1.70 [sec] | Valid Loss: 686.086, Valid Time: 0.66 [sec]
::::::::::::::::::::::::: best loss: 686.086 :::::::::::::::::::::::::
::::::::::::::::::::::::: Exp.3 :::::::::::::::::::::::::
2022-09-21 09:03:05.508555 | Epoch: 1 | Train Loss: 864.096, Train 

### 検索の有用性を比較（ST-ED vs STE-ED vs STE-ED-S）

In [14]:
TRAFFIC_CONV = 64
TRAFFIC_HIDDEN = 128
TRAFFIC_LSTM_LAYERS = 4
TRAFFIC_KERNEL = (7,5)

SEARCH_CONV = 64
SEARCH_HIDDEN = 128
SEARCH_LSTM_LAYERS = 4
SEARCH_KERNEL = (7,5)

UNSPEC_SEARCH_HIDDEN = 64
UNSPEC_SEARCH_KERNEL = 5

DATETIME_EMB = 32
ROAD_EMB = 16

FC_EMB = 32

PREDICTION_HORIZON = 24

In [18]:
time_step = 7 * 24
space_window = (-2, 2)

#### ST-ED

In [51]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_both, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_both, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/ST-ED_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:15:10.685435 | Epoch: 1 | Train Loss: 884.998, Train Time: 4.37 [sec] | Valid Loss: 862.037, Valid Time: 0.91 [sec]
2022-09-29 10:15:16.003705 | Epoch: 2 | Train Loss: 728.254, Train Time: 4.38 [sec] | Valid Loss: 723.285, Valid Time: 0.91 [sec]
2022-09-29 10:15:21.352083 | Epoch: 3 | Train Loss: 660.152, Train Time: 4.40 [sec] | Valid Loss: 659.807, Valid Time: 0.91 [sec]
2022-09-29 10:15:26.851966 | Epoch: 4 | Train Loss: 504.893, Train Time: 4.55 [sec] | Valid Loss: 442.367, Valid Time: 0.91 [sec]
2022-09-29 10:15:32.259302 | Epoch: 5 | Train Loss: 322.482, Train Time: 4.46 [sec] | Valid Loss: 299.131, Valid Time: 0.91 [sec]
2022-09-29 10:15:37.686665 | Epoch: 6 | Train Loss: 224.775, Train Time: 4.47 [sec] | Valid Loss: 222.983, Valid Time: 0.91 [sec]
2022-09-29 10:15:43.123238 | Epoch: 7 | Train Loss: 178.454, Train Time: 4.48 [sec] | Valid Loss: 186.756, Valid Time: 0.91 [sec]
2022-09-29 10:15:48.571414 | Epoch: 8 | Train Loss: 155.426, Train Time: 4.49 [sec] | Vali

#### ST-ED-S

In [52]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_context, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_context, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/ST-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:24:19.423752 | Epoch: 1 | Train Loss: 884.047, Train Time: 4.45 [sec] | Valid Loss: 858.919, Valid Time: 0.91 [sec]
2022-09-29 10:24:24.956937 | Epoch: 2 | Train Loss: 722.274, Train Time: 4.46 [sec] | Valid Loss: 712.109, Valid Time: 1.04 [sec]
2022-09-29 10:24:30.386196 | Epoch: 3 | Train Loss: 626.582, Train Time: 4.47 [sec] | Valid Loss: 574.777, Valid Time: 0.91 [sec]
2022-09-29 10:24:35.836868 | Epoch: 4 | Train Loss: 439.280, Train Time: 4.50 [sec] | Valid Loss: 386.793, Valid Time: 0.91 [sec]
2022-09-29 10:24:41.302459 | Epoch: 5 | Train Loss: 306.412, Train Time: 4.51 [sec] | Valid Loss: 272.567, Valid Time: 0.92 [sec]
2022-09-29 10:24:46.890899 | Epoch: 6 | Train Loss: 224.691, Train Time: 4.63 [sec] | Valid Loss: 230.020, Valid Time: 0.91 [sec]
2022-09-29 10:24:52.359531 | Epoch: 7 | Train Loss: 177.574, Train Time: 4.51 [sec] | Valid Loss: 187.574, Valid Time: 0.92 [sec]
2022-09-29 10:24:57.819853 | Epoch: 8 | Train Loss: 152.402, Train Time: 4.50 [sec] | Vali

#### ST-ED-S1

In [52]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_context, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_context, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/ST-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:24:19.423752 | Epoch: 1 | Train Loss: 884.047, Train Time: 4.45 [sec] | Valid Loss: 858.919, Valid Time: 0.91 [sec]
2022-09-29 10:24:24.956937 | Epoch: 2 | Train Loss: 722.274, Train Time: 4.46 [sec] | Valid Loss: 712.109, Valid Time: 1.04 [sec]
2022-09-29 10:24:30.386196 | Epoch: 3 | Train Loss: 626.582, Train Time: 4.47 [sec] | Valid Loss: 574.777, Valid Time: 0.91 [sec]
2022-09-29 10:24:35.836868 | Epoch: 4 | Train Loss: 439.280, Train Time: 4.50 [sec] | Valid Loss: 386.793, Valid Time: 0.91 [sec]
2022-09-29 10:24:41.302459 | Epoch: 5 | Train Loss: 306.412, Train Time: 4.51 [sec] | Valid Loss: 272.567, Valid Time: 0.92 [sec]
2022-09-29 10:24:46.890899 | Epoch: 6 | Train Loss: 224.691, Train Time: 4.63 [sec] | Valid Loss: 230.020, Valid Time: 0.91 [sec]
2022-09-29 10:24:52.359531 | Epoch: 7 | Train Loss: 177.574, Train Time: 4.51 [sec] | Valid Loss: 187.574, Valid Time: 0.92 [sec]
2022-09-29 10:24:57.819853 | Epoch: 8 | Train Loss: 152.402, Train Time: 4.50 [sec] | Vali

#### STE-ED

In [50]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_search, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_search, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/STE-ED_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:05:55.620660 | Epoch: 1 | Train Loss: 887.670, Train Time: 4.27 [sec] | Valid Loss: 867.993, Valid Time: 0.90 [sec]
2022-09-29 10:06:00.940534 | Epoch: 2 | Train Loss: 732.711, Train Time: 4.39 [sec] | Valid Loss: 725.063, Valid Time: 0.90 [sec]
2022-09-29 10:06:06.189051 | Epoch: 3 | Train Loss: 624.695, Train Time: 4.31 [sec] | Valid Loss: 576.753, Valid Time: 0.90 [sec]
2022-09-29 10:06:11.461485 | Epoch: 4 | Train Loss: 443.205, Train Time: 4.33 [sec] | Valid Loss: 391.670, Valid Time: 0.90 [sec]
2022-09-29 10:06:16.872798 | Epoch: 5 | Train Loss: 302.375, Train Time: 4.47 [sec] | Valid Loss: 297.201, Valid Time: 0.90 [sec]
2022-09-29 10:06:22.165748 | Epoch: 6 | Train Loss: 232.168, Train Time: 4.35 [sec] | Valid Loss: 229.357, Valid Time: 0.90 [sec]
2022-09-29 10:06:27.490039 | Epoch: 7 | Train Loss: 190.038, Train Time: 4.38 [sec] | Valid Loss: 200.445, Valid Time: 0.91 [sec]
2022-09-29 10:06:32.841902 | Epoch: 8 | Train Loss: 169.546, Train Time: 4.40 [sec] | Vali

#### STE-ED-S

In [49]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/STE-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 09:55:16.926786 | Epoch: 1 | Train Loss: 886.051, Train Time: 6.26 [sec] | Valid Loss: 863.103, Valid Time: 0.90 [sec]
2022-09-29 09:55:22.106408 | Epoch: 2 | Train Loss: 725.203, Train Time: 4.25 [sec] | Valid Loss: 712.954, Valid Time: 0.90 [sec]
2022-09-29 09:55:27.312915 | Epoch: 3 | Train Loss: 637.516, Train Time: 4.27 [sec] | Valid Loss: 609.430, Valid Time: 0.90 [sec]
2022-09-29 09:55:32.678569 | Epoch: 4 | Train Loss: 466.276, Train Time: 4.42 [sec] | Valid Loss: 416.088, Valid Time: 0.90 [sec]
2022-09-29 09:55:37.937783 | Epoch: 5 | Train Loss: 329.638, Train Time: 4.32 [sec] | Valid Loss: 301.526, Valid Time: 0.90 [sec]
2022-09-29 09:55:43.205038 | Epoch: 6 | Train Loss: 244.340, Train Time: 4.33 [sec] | Valid Loss: 241.454, Valid Time: 0.90 [sec]
2022-09-29 09:55:48.505302 | Epoch: 7 | Train Loss: 184.896, Train Time: 4.35 [sec] | Valid Loss: 194.676, Valid Time: 0.91 [sec]
2022-09-29 09:55:53.931883 | Epoch: 8 | Train Loss: 152.953, Train Time: 4.49 [sec] | Vali

### 検索の有用性を比較2 （T-ED vs T-ED-S vs TE-ED vs STE-ED-S）

In [19]:
TRAFFIC_CONV = 64
TRAFFIC_HIDDEN = 128
TRAFFIC_LSTM_LAYERS = 4
TRAFFIC_KERNEL = (7,5)

SEARCH_CONV = 64
SEARCH_HIDDEN = 128
SEARCH_LSTM_LAYERS = 4
SEARCH_KERNEL = (7,5)

UNSPEC_SEARCH_HIDDEN = 1
UNSPEC_SEARCH_KERNEL = 5

DATETIME_EMB = 32
ROAD_EMB = 16

FC_EMB = 32

PREDICTION_HORIZON = 24

In [20]:
time_step = 7 * 24
space_window = (-2, 2)

#### T-ED

In [72]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_both, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_both, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/T-ED_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = T_EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:54:32.466083 | Epoch: 1 | Train Loss: 882.171, Train Time: 4.43 [sec] | Valid Loss: 859.125, Valid Time: 0.90 [sec]
2022-09-29 10:54:37.834574 | Epoch: 2 | Train Loss: 728.928, Train Time: 4.44 [sec] | Valid Loss: 725.766, Valid Time: 0.90 [sec]
2022-09-29 10:54:43.223120 | Epoch: 3 | Train Loss: 664.911, Train Time: 4.44 [sec] | Valid Loss: 678.536, Valid Time: 0.91 [sec]
2022-09-29 10:54:48.635379 | Epoch: 4 | Train Loss: 555.895, Train Time: 4.47 [sec] | Valid Loss: 532.849, Valid Time: 0.90 [sec]
2022-09-29 10:54:54.204372 | Epoch: 5 | Train Loss: 405.315, Train Time: 4.62 [sec] | Valid Loss: 335.582, Valid Time: 0.91 [sec]
2022-09-29 10:54:59.649298 | Epoch: 6 | Train Loss: 258.806, Train Time: 4.50 [sec] | Valid Loss: 244.153, Valid Time: 0.91 [sec]
2022-09-29 10:55:05.094794 | Epoch: 7 | Train Loss: 189.935, Train Time: 4.50 [sec] | Valid Loss: 189.990, Valid Time: 0.91 [sec]
2022-09-29 10:55:10.554095 | Epoch: 8 | Train Loss: 159.686, Train Time: 4.51 [sec] | Vali

#### T-ED-S

In [73]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_context, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_context, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/T-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = T_EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 11:03:51.149311 | Epoch: 1 | Train Loss: 882.128, Train Time: 4.70 [sec] | Valid Loss: 858.993, Valid Time: 0.91 [sec]
2022-09-29 11:03:56.689307 | Epoch: 2 | Train Loss: 728.699, Train Time: 4.60 [sec] | Valid Loss: 725.349, Valid Time: 0.91 [sec]
2022-09-29 11:04:02.243647 | Epoch: 3 | Train Loss: 664.145, Train Time: 4.60 [sec] | Valid Loss: 676.062, Valid Time: 0.92 [sec]
2022-09-29 11:04:07.819692 | Epoch: 4 | Train Loss: 526.985, Train Time: 4.62 [sec] | Valid Loss: 459.269, Valid Time: 0.92 [sec]
2022-09-29 11:04:13.399050 | Epoch: 5 | Train Loss: 340.812, Train Time: 4.63 [sec] | Valid Loss: 304.058, Valid Time: 0.92 [sec]
2022-09-29 11:04:18.976794 | Epoch: 6 | Train Loss: 244.564, Train Time: 4.62 [sec] | Valid Loss: 240.228, Valid Time: 0.91 [sec]
2022-09-29 11:04:24.692566 | Epoch: 7 | Train Loss: 194.637, Train Time: 4.76 [sec] | Valid Loss: 189.922, Valid Time: 0.92 [sec]
2022-09-29 11:04:30.284779 | Epoch: 8 | Train Loss: 164.326, Train Time: 4.64 [sec] | Vali

#### TE-ED

In [74]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_search, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_search, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/TE-ED_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = T_EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 11:13:12.471953 | Epoch: 1 | Train Loss: 884.030, Train Time: 4.59 [sec] | Valid Loss: 864.645, Valid Time: 0.91 [sec]
2022-09-29 11:13:18.018253 | Epoch: 2 | Train Loss: 733.916, Train Time: 4.60 [sec] | Valid Loss: 728.690, Valid Time: 0.92 [sec]
2022-09-29 11:13:23.581549 | Epoch: 3 | Train Loss: 664.513, Train Time: 4.61 [sec] | Valid Loss: 661.813, Valid Time: 0.91 [sec]
2022-09-29 11:13:29.284975 | Epoch: 4 | Train Loss: 521.869, Train Time: 4.75 [sec] | Valid Loss: 475.744, Valid Time: 0.92 [sec]
2022-09-29 11:13:34.868285 | Epoch: 5 | Train Loss: 374.212, Train Time: 4.63 [sec] | Valid Loss: 346.918, Valid Time: 0.91 [sec]
2022-09-29 11:13:40.431393 | Epoch: 6 | Train Loss: 306.846, Train Time: 4.61 [sec] | Valid Loss: 317.289, Valid Time: 0.92 [sec]
2022-09-29 11:13:46.036676 | Epoch: 7 | Train Loss: 250.669, Train Time: 4.65 [sec] | Valid Loss: 260.428, Valid Time: 0.92 [sec]
2022-09-29 11:13:51.640510 | Epoch: 8 | Train Loss: 193.898, Train Time: 4.65 [sec] | Vali

#### TE-ED-S

In [75]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/TE-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = T_EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 11:22:34.164888 | Epoch: 1 | Train Loss: 884.054, Train Time: 4.57 [sec] | Valid Loss: 864.634, Valid Time: 0.91 [sec]
2022-09-29 11:22:39.857566 | Epoch: 2 | Train Loss: 733.647, Train Time: 4.75 [sec] | Valid Loss: 728.150, Valid Time: 0.91 [sec]
2022-09-29 11:22:45.426348 | Epoch: 3 | Train Loss: 664.270, Train Time: 4.62 [sec] | Valid Loss: 682.457, Valid Time: 0.91 [sec]
2022-09-29 11:22:50.994573 | Epoch: 4 | Train Loss: 546.073, Train Time: 4.61 [sec] | Valid Loss: 490.116, Valid Time: 0.92 [sec]
2022-09-29 11:22:56.588350 | Epoch: 5 | Train Loss: 383.262, Train Time: 4.64 [sec] | Valid Loss: 399.892, Valid Time: 0.92 [sec]
2022-09-29 11:23:02.181329 | Epoch: 6 | Train Loss: 311.399, Train Time: 4.64 [sec] | Valid Loss: 302.940, Valid Time: 0.92 [sec]
2022-09-29 11:23:07.778343 | Epoch: 7 | Train Loss: 244.765, Train Time: 4.64 [sec] | Valid Loss: 244.791, Valid Time: 0.92 [sec]
2022-09-29 11:23:13.500575 | Epoch: 8 | Train Loss: 199.404, Train Time: 4.77 [sec] | Vali

### 時間あり検索, 時間なし検索の有用性を比較（STE-ED vs STE-ED-S(1) vs STE-ED-S(2) vs STE-ED-S）

In [28]:
TRAFFIC_CONV = 64
TRAFFIC_HIDDEN = 128
TRAFFIC_LSTM_LAYERS = 4
TRAFFIC_KERNEL = (7,5)

SEARCH_CONV = 64
SEARCH_HIDDEN = 128
SEARCH_LSTM_LAYERS = 4
SEARCH_KERNEL = (7,5)

UNSPEC_SEARCH_HIDDEN = 64
UNSPEC_SEARCH_KERNEL = 5

DATETIME_EMB = 32
ROAD_EMB = 16

FC_EMB = 32

PREDICTION_HORIZON = 24

In [29]:
time_step = 7 * 24
space_window = (-2, 2)

#### STE-ED

In [50]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_search, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_search, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/STE-ED_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:05:55.620660 | Epoch: 1 | Train Loss: 887.670, Train Time: 4.27 [sec] | Valid Loss: 867.993, Valid Time: 0.90 [sec]
2022-09-29 10:06:00.940534 | Epoch: 2 | Train Loss: 732.711, Train Time: 4.39 [sec] | Valid Loss: 725.063, Valid Time: 0.90 [sec]
2022-09-29 10:06:06.189051 | Epoch: 3 | Train Loss: 624.695, Train Time: 4.31 [sec] | Valid Loss: 576.753, Valid Time: 0.90 [sec]
2022-09-29 10:06:11.461485 | Epoch: 4 | Train Loss: 443.205, Train Time: 4.33 [sec] | Valid Loss: 391.670, Valid Time: 0.90 [sec]
2022-09-29 10:06:16.872798 | Epoch: 5 | Train Loss: 302.375, Train Time: 4.47 [sec] | Valid Loss: 297.201, Valid Time: 0.90 [sec]
2022-09-29 10:06:22.165748 | Epoch: 6 | Train Loss: 232.168, Train Time: 4.35 [sec] | Valid Loss: 229.357, Valid Time: 0.90 [sec]
2022-09-29 10:06:27.490039 | Epoch: 7 | Train Loss: 190.038, Train Time: 4.38 [sec] | Valid Loss: 200.445, Valid Time: 0.91 [sec]
2022-09-29 10:06:32.841902 | Epoch: 8 | Train Loss: 169.546, Train Time: 4.40 [sec] | Vali

#### STE-ED-S(1)

In [30]:
X_tr_wo_only_unspec = zeroing_feature(X_tr, [3])
X_va_wo_only_unspec = zeroing_feature(X_va, [3])

In [32]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_only_unspec, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_only_unspec, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/STE-ED-S(1)_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-30 04:05:55.110108 | Epoch: 1 | Train Loss: 887.391, Train Time: 4.31 [sec] | Valid Loss: 867.334, Valid Time: 0.90 [sec]
2022-09-30 04:06:00.363788 | Epoch: 2 | Train Loss: 732.176, Train Time: 4.32 [sec] | Valid Loss: 724.741, Valid Time: 0.91 [sec]
2022-09-30 04:06:05.797252 | Epoch: 3 | Train Loss: 662.155, Train Time: 4.48 [sec] | Valid Loss: 675.820, Valid Time: 0.91 [sec]
2022-09-30 04:06:11.117679 | Epoch: 4 | Train Loss: 508.926, Train Time: 4.37 [sec] | Valid Loss: 441.591, Valid Time: 0.91 [sec]
2022-09-30 04:06:16.472383 | Epoch: 5 | Train Loss: 312.836, Train Time: 4.40 [sec] | Valid Loss: 280.216, Valid Time: 0.92 [sec]
2022-09-30 04:06:21.857444 | Epoch: 6 | Train Loss: 213.169, Train Time: 4.42 [sec] | Valid Loss: 210.812, Valid Time: 0.92 [sec]
2022-09-30 04:06:27.314044 | Epoch: 7 | Train Loss: 167.769, Train Time: 4.49 [sec] | Valid Loss: 183.680, Valid Time: 0.93 [sec]
2022-09-30 04:06:32.957978 | Epoch: 8 | Train Loss: 145.134, Train Time: 4.68 [sec] | Vali

#### STE-ED-S(2)

In [33]:
X_tr_wo_only_spec = zeroing_feature(X_tr, [2])
X_va_wo_only_spec = zeroing_feature(X_va, [2])

In [34]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_only_spec, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_only_spec, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/STE-ED-S(2)_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-30 04:15:10.871574 | Epoch: 1 | Train Loss: 886.016, Train Time: 4.51 [sec] | Valid Loss: 863.025, Valid Time: 0.93 [sec]
2022-09-30 04:15:16.349551 | Epoch: 2 | Train Loss: 725.260, Train Time: 4.52 [sec] | Valid Loss: 713.048, Valid Time: 0.93 [sec]
2022-09-30 04:15:21.840242 | Epoch: 3 | Train Loss: 640.245, Train Time: 4.53 [sec] | Valid Loss: 638.017, Valid Time: 0.93 [sec]
2022-09-30 04:15:27.362095 | Epoch: 4 | Train Loss: 502.111, Train Time: 4.56 [sec] | Valid Loss: 432.038, Valid Time: 0.93 [sec]
2022-09-30 04:15:32.872240 | Epoch: 5 | Train Loss: 339.246, Train Time: 4.55 [sec] | Valid Loss: 300.763, Valid Time: 0.92 [sec]
2022-09-30 04:15:38.517178 | Epoch: 6 | Train Loss: 238.763, Train Time: 4.68 [sec] | Valid Loss: 247.649, Valid Time: 0.93 [sec]
2022-09-30 04:15:44.043411 | Epoch: 7 | Train Loss: 171.307, Train Time: 4.56 [sec] | Valid Loss: 196.845, Valid Time: 0.93 [sec]
2022-09-30 04:15:49.556605 | Epoch: 8 | Train Loss: 134.212, Train Time: 4.55 [sec] | Vali

#### STE-ED-S

In [49]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/STE-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 09:55:16.926786 | Epoch: 1 | Train Loss: 886.051, Train Time: 6.26 [sec] | Valid Loss: 863.103, Valid Time: 0.90 [sec]
2022-09-29 09:55:22.106408 | Epoch: 2 | Train Loss: 725.203, Train Time: 4.25 [sec] | Valid Loss: 712.954, Valid Time: 0.90 [sec]
2022-09-29 09:55:27.312915 | Epoch: 3 | Train Loss: 637.516, Train Time: 4.27 [sec] | Valid Loss: 609.430, Valid Time: 0.90 [sec]
2022-09-29 09:55:32.678569 | Epoch: 4 | Train Loss: 466.276, Train Time: 4.42 [sec] | Valid Loss: 416.088, Valid Time: 0.90 [sec]
2022-09-29 09:55:37.937783 | Epoch: 5 | Train Loss: 329.638, Train Time: 4.32 [sec] | Valid Loss: 301.526, Valid Time: 0.90 [sec]
2022-09-29 09:55:43.205038 | Epoch: 6 | Train Loss: 244.340, Train Time: 4.33 [sec] | Valid Loss: 241.454, Valid Time: 0.90 [sec]
2022-09-29 09:55:48.505302 | Epoch: 7 | Train Loss: 184.896, Train Time: 4.35 [sec] | Valid Loss: 194.676, Valid Time: 0.91 [sec]
2022-09-29 09:55:53.931883 | Epoch: 8 | Train Loss: 152.953, Train Time: 4.49 [sec] | Vali

### Input Time Steps, Prediction Time Stepsを変えてrobustnessを検証

In [15]:
TRAFFIC_CONV = 64
TRAFFIC_HIDDEN = 128
TRAFFIC_LSTM_LAYERS = 4
TRAFFIC_KERNEL = (7,5)

SEARCH_CONV = 64
SEARCH_HIDDEN = 128
SEARCH_LSTM_LAYERS = 4
SEARCH_KERNEL = (7,5)

UNSPEC_SEARCH_HIDDEN = 64
UNSPEC_SEARCH_KERNEL = 5

DATETIME_EMB = 32
ROAD_EMB = 16

FC_EMB = 32

PREDICTION_HORIZON = 24

In [16]:
time_step = 7 * 24
space_window = (-2, 2)

#### T-ED

In [20]:
n_epochs = 100
random_seed = 1000

input_time_steps = [1*24, 3*24, 7*24]

trainer_list = []

for in_step in input_time_steps:
    print('='*40, f'Input Steps: {in_step//24} day', '='*40)
    
    dataset_train = STDataset(X_tr, y_tr, time_step=in_step, space_window=space_window)
    dataset_valid = STDataset(X_va, y_va, time_step=in_step, space_window=space_window)

    model_name = f'dropout0/T-ED_from{in_step}_RS{random_seed}'
    log_path = f'./logs/{model_name}.log'

    fix_seed(random_seed)
    
    model = LSTMOnlyEncoderDecoder(lstm_dropout=0)
    trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                    model_name, log_path, random_state=random_seed)
    trainer_list.append(trainer)

    print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

======================================== Input Steps: 1 day ========================================
2022-10-03 06:08:07.271160 | Epoch: 1 | Train Loss: 915.408, Train Time: 1.77 [sec] | Valid Loss: 914.742, Valid Time: 0.45 [sec]
2022-10-03 06:08:09.516387 | Epoch: 2 | Train Loss: 811.877, Train Time: 1.77 [sec] | Valid Loss: 782.491, Valid Time: 0.45 [sec]
2022-10-03 06:08:11.759112 | Epoch: 3 | Train Loss: 707.987, Train Time: 1.77 [sec] | Valid Loss: 710.923, Valid Time: 0.45 [sec]
2022-10-03 06:08:14.005934 | Epoch: 4 | Train Loss: 655.280, Train Time: 1.77 [sec] | Valid Loss: 631.605, Valid Time: 0.45 [sec]
2022-10-03 06:08:16.378770 | Epoch: 5 | Train Loss: 533.986, Train Time: 1.76 [sec] | Valid Loss: 495.105, Valid Time: 0.58 [sec]
2022-10-03 06:08:18.622576 | Epoch: 6 | Train Loss: 418.240, Train Time: 1.77 [sec] | Valid Loss: 396.288, Valid Time: 0.45 [sec]
2022-10-03 06:08:20.874783 | Epoch: 7 | Train Loss: 314.909, Train Time: 1.77 [sec] | Valid Loss: 297.590, Valid Time: 

In [21]:
input_time_steps = [1*24, 3*24, 7*24]

for in_step, trainer in zip(input_time_steps, trainer_list):
    print(f'Input: {in_step} hours | Loss: {min(trainer.val_losses):.3f}')

Input: 24 hours | Loss: 119.212
Input: 72 hours | Loss: 130.941
Input: 168 hours | Loss: 116.641


#### ST-ED

In [22]:
n_epochs = 100
random_seed = 1000

input_time_steps = [1*24, 3*24, 7*24]

trainer_list = []

for in_step in input_time_steps:
    print('='*40, f'Input Steps: {in_step//24} day', '='*40)
    
    dataset_train = STDataset(X_tr, y_tr, time_step=in_step, space_window=space_window)
    dataset_valid = STDataset(X_va, y_va, time_step=in_step, space_window=space_window)

    model_name = f'dropout0/ST-ED_from{in_step}_RS{random_seed}'
    log_path = f'./logs/{model_name}.log'

    fix_seed(random_seed)

    model = CNNLSTMEncoderDecoder(lstm_dropout=0)
    trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                    model_name, log_path, random_state=random_seed)
    trainer_list.append(trainer)

    print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

======================================== Input Steps: 1 day ========================================
2022-10-03 06:23:45.058407 | Epoch: 1 | Train Loss: 912.560, Train Time: 3.56 [sec] | Valid Loss: 909.080, Valid Time: 0.48 [sec]
2022-10-03 06:23:47.397098 | Epoch: 2 | Train Loss: 806.006, Train Time: 1.84 [sec] | Valid Loss: 777.604, Valid Time: 0.48 [sec]
2022-10-03 06:23:49.773174 | Epoch: 3 | Train Loss: 705.603, Train Time: 1.87 [sec] | Valid Loss: 709.864, Valid Time: 0.48 [sec]
2022-10-03 06:23:52.201737 | Epoch: 4 | Train Loss: 632.625, Train Time: 1.92 [sec] | Valid Loss: 589.031, Valid Time: 0.48 [sec]
2022-10-03 06:23:54.770065 | Epoch: 5 | Train Loss: 479.306, Train Time: 1.93 [sec] | Valid Loss: 440.558, Valid Time: 0.61 [sec]
2022-10-03 06:23:57.188620 | Epoch: 6 | Train Loss: 339.930, Train Time: 1.92 [sec] | Valid Loss: 340.492, Valid Time: 0.48 [sec]
2022-10-03 06:23:59.621105 | Epoch: 7 | Train Loss: 254.863, Train Time: 1.93 [sec] | Valid Loss: 258.254, Valid Time: 

In [23]:
for in_step, trainer in zip(input_time_steps, trainer_list):
    print(f'Input: {in_step} hours | Loss: {min(trainer.val_losses):.3f}')

Input: 24 hours | Loss: 116.287
Input: 72 hours | Loss: 119.579
Input: 168 hours | Loss: 116.477


#### TE-ED

In [24]:
n_epochs = 100
random_seed = 1000

input_time_steps = [1*24, 3*24, 7*24]

trainer_list = []

for in_step in input_time_steps:
    print('='*40, f'Input Steps: {in_step//24} day', '='*40)
    
    dataset_train = STDataset(X_tr, y_tr, time_step=in_step, space_window=space_window)
    dataset_valid = STDataset(X_va, y_va, time_step=in_step, space_window=space_window)

    model_name = f'dropout0/TE-ED_from{in_step}_RS{random_seed}'
    log_path = f'./logs/{model_name}.log'

    fix_seed(random_seed)

    model = LSTMEmbeddingEncoderDecoder(lstm_dropout=0)
    trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                    model_name, log_path, random_state=random_seed)
    trainer_list.append(trainer)

    print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

======================================== Input Steps: 1 day ========================================
2022-10-03 06:40:09.651369 | Epoch: 1 | Train Loss: 915.655, Train Time: 2.07 [sec] | Valid Loss: 917.918, Valid Time: 0.62 [sec]
2022-10-03 06:40:12.265979 | Epoch: 2 | Train Loss: 820.587, Train Time: 1.98 [sec] | Valid Loss: 795.300, Valid Time: 0.62 [sec]
2022-10-03 06:40:14.887358 | Epoch: 3 | Train Loss: 715.592, Train Time: 1.98 [sec] | Valid Loss: 713.081, Valid Time: 0.62 [sec]
2022-10-03 06:40:17.518799 | Epoch: 4 | Train Loss: 664.023, Train Time: 1.99 [sec] | Valid Loss: 652.479, Valid Time: 0.62 [sec]
2022-10-03 06:40:20.273638 | Epoch: 5 | Train Loss: 565.405, Train Time: 2.11 [sec] | Valid Loss: 535.263, Valid Time: 0.62 [sec]
2022-10-03 06:40:22.888455 | Epoch: 6 | Train Loss: 447.873, Train Time: 1.97 [sec] | Valid Loss: 421.143, Valid Time: 0.62 [sec]
2022-10-03 06:40:25.506581 | Epoch: 7 | Train Loss: 361.711, Train Time: 1.97 [sec] | Valid Loss: 346.025, Valid Time: 

In [25]:
for in_step, trainer in zip(input_time_steps, trainer_list):
    print(f'Input: {in_step} hours | Loss: {min(trainer.val_losses):.3f}')

Input: 24 hours | Loss: 110.545
Input: 72 hours | Loss: 109.089
Input: 168 hours | Loss: 108.604


#### STE-ED

In [26]:
n_epochs = 100
random_seed = 1000

input_time_steps = [1*24, 3*24, 7*24]

trainer_list = []

for in_step in input_time_steps:
    print('='*40, f'Input Steps: {in_step//24} day', '='*40)
    
    dataset_train = STDataset(X_tr, y_tr, time_step=in_step, space_window=space_window)
    dataset_valid = STDataset(X_va, y_va, time_step=in_step, space_window=space_window)

    model_name = f'dropout0/STE-ED_from{in_step}_RS{random_seed}'
    log_path = f'./logs/{model_name}.log'

    fix_seed(random_seed)

    model = EncoderDecoder(lstm_dropout=0, include_search=False)
    trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                    model_name, log_path, random_state=random_seed)
    trainer_list.append(trainer)

    print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

======================================== Input Steps: 1 day ========================================
2022-10-03 06:57:35.174890 | Epoch: 1 | Train Loss: 903.535, Train Time: 2.02 [sec] | Valid Loss: 887.413, Valid Time: 0.76 [sec]
2022-10-03 06:57:37.896256 | Epoch: 2 | Train Loss: 776.988, Train Time: 2.07 [sec] | Valid Loss: 746.101, Valid Time: 0.63 [sec]
2022-10-03 06:57:40.622634 | Epoch: 3 | Train Loss: 685.437, Train Time: 2.07 [sec] | Valid Loss: 692.684, Valid Time: 0.63 [sec]
2022-10-03 06:57:43.352939 | Epoch: 4 | Train Loss: 582.807, Train Time: 2.07 [sec] | Valid Loss: 530.794, Valid Time: 0.63 [sec]
2022-10-03 06:57:46.091167 | Epoch: 5 | Train Loss: 426.159, Train Time: 2.08 [sec] | Valid Loss: 383.590, Valid Time: 0.63 [sec]
2022-10-03 06:57:48.959860 | Epoch: 6 | Train Loss: 288.864, Train Time: 2.20 [sec] | Valid Loss: 262.013, Valid Time: 0.63 [sec]
2022-10-03 06:57:51.694395 | Epoch: 7 | Train Loss: 204.602, Train Time: 2.08 [sec] | Valid Loss: 219.330, Valid Time: 

In [27]:
for in_step, trainer in zip(input_time_steps, trainer_list):
    print(f'Input: {in_step} hours | Loss: {min(trainer.val_losses):.3f}')

Input: 24 hours | Loss: 109.483
Input: 72 hours | Loss: 113.009
Input: 168 hours | Loss: 113.668


#### STE-ED-S

In [28]:
n_epochs = 100
random_seed = 1000

input_time_steps = [1*24, 3*24, 7*24]

trainer_list = []

for in_step in input_time_steps:
    print('='*40, f'Input Steps: {in_step//24} day', '='*40)
    
    dataset_train = STDataset(X_tr, y_tr, time_step=in_step, space_window=space_window)
    dataset_valid = STDataset(X_va, y_va, time_step=in_step, space_window=space_window)

    model_name = f'dropout0/STE-ED-S_from{in_step}_RS{random_seed}'
    log_path = f'./logs/{model_name}.log'

    fix_seed(random_seed)

    model = EncoderDecoder(lstm_dropout=0)
    trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                    model_name, log_path, random_state=random_seed)
    trainer_list.append(trainer)

    print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

======================================== Input Steps: 1 day ========================================
2022-10-03 07:15:24.341738 | Epoch: 1 | Train Loss: 886.309, Train Time: 2.72 [sec] | Valid Loss: 841.383, Valid Time: 0.75 [sec]
2022-10-03 07:15:27.829885 | Epoch: 2 | Train Loss: 722.931, Train Time: 2.72 [sec] | Valid Loss: 696.468, Valid Time: 0.74 [sec]
2022-10-03 07:15:31.581262 | Epoch: 3 | Train Loss: 635.044, Train Time: 2.97 [sec] | Valid Loss: 592.886, Valid Time: 0.74 [sec]
2022-10-03 07:15:35.259350 | Epoch: 4 | Train Loss: 463.704, Train Time: 2.90 [sec] | Valid Loss: 402.492, Valid Time: 0.74 [sec]
2022-10-03 07:15:38.919781 | Epoch: 5 | Train Loss: 330.793, Train Time: 2.88 [sec] | Valid Loss: 301.563, Valid Time: 0.74 [sec]
2022-10-03 07:15:42.584389 | Epoch: 6 | Train Loss: 241.280, Train Time: 2.87 [sec] | Valid Loss: 247.170, Valid Time: 0.75 [sec]
2022-10-03 07:15:46.377082 | Epoch: 7 | Train Loss: 171.987, Train Time: 3.01 [sec] | Valid Loss: 181.865, Valid Time: 

In [29]:
input_time_steps = [1*24, 3*24, 7*24]

for in_step, trainer in zip(input_time_steps, trainer_list):
    print(f'Input: {in_step} hours | Loss: {min(trainer.val_losses):.3f}')

Input: 24 hours | Loss: 97.886
Input: 72 hours | Loss: 103.922
Input: 168 hours | Loss: 101.173


### 検索あり

In [18]:
n_epochs = 100

lstm_layers_list = range(1, 4+1)
trainer_list = []

for lstm_layers in lstm_layers_list:
    model_name = f'dropout0/EncDec_LSTM{lstm_layers}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'LSTM layers: {lstm_layers}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = EncoderDecoder(lstm_dropout=0)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(LSTM layers = {lstm_layers}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

======================================== LSTM layers: 1 ========================================
2022-09-21 09:05:17.287951 | Epoch: 1 | Train Loss: 864.096, Train Time: 1.72 [sec] | Valid Loss: 809.587, Valid Time: 0.57 [sec]
2022-09-21 09:05:19.576078 | Epoch: 2 | Train Loss: 687.757, Train Time: 1.70 [sec] | Valid Loss: 686.086, Valid Time: 0.58 [sec]
2022-09-21 09:05:21.946317 | Epoch: 3 | Train Loss: 603.147, Train Time: 1.79 [sec] | Valid Loss: 578.126, Valid Time: 0.58 [sec]
2022-09-21 09:05:24.231060 | Epoch: 4 | Train Loss: 424.626, Train Time: 1.70 [sec] | Valid Loss: 365.330, Valid Time: 0.58 [sec]
2022-09-21 09:05:26.599020 | Epoch: 5 | Train Loss: 278.149, Train Time: 1.70 [sec] | Valid Loss: 260.981, Valid Time: 0.66 [sec]
2022-09-21 09:05:28.881264 | Epoch: 6 | Train Loss: 205.269, Train Time: 1.70 [sec] | Valid Loss: 213.722, Valid Time: 0.58 [sec]
2022-09-21 09:05:31.161376 | Epoch: 7 | Train Loss: 161.365, Train Time: 1.70 [sec] | Valid Loss: 218.580, Valid Time: 0.58

In [19]:
{i+1: min(trainer.val_losses) for i, trainer in enumerate(trainer_list)}

{1: 106.1190679640997,
 2: 107.17107118879046,
 3: 104.1589845929827,
 4: 103.67929912748791}

In [43]:
n_epochs = 100

random_seeds = [32, 1000]
trainer_list = []

for random_seed in random_seeds:
    fix_seed(random_seed)
    
    model_name = f'dropout0/STE-ED-S_LSTM4_RS{random_seed}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Random Seed: {random_seed}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = 4
    
    model = EncoderDecoder(lstm_dropout=0)
    trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
    print('='*40, f'(Random Seed = {random_seed}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

======================================== Random Seed: 32 ========================================
2022-09-22 14:13:09.752071 | Epoch: 1 | Train Loss: 880.177, Train Time: 4.30 [sec] | Valid Loss: 852.806, Valid Time: 0.90 [sec]
2022-09-22 14:13:15.106189 | Epoch: 2 | Train Loss: 717.261, Train Time: 4.42 [sec] | Valid Loss: 705.951, Valid Time: 0.91 [sec]
2022-09-22 14:13:20.361302 | Epoch: 3 | Train Loss: 629.719, Train Time: 4.31 [sec] | Valid Loss: 622.474, Valid Time: 0.91 [sec]
2022-09-22 14:13:25.646085 | Epoch: 4 | Train Loss: 473.527, Train Time: 4.34 [sec] | Valid Loss: 399.834, Valid Time: 0.91 [sec]
2022-09-22 14:13:30.944898 | Epoch: 5 | Train Loss: 310.746, Train Time: 4.35 [sec] | Valid Loss: 276.185, Valid Time: 0.91 [sec]
2022-09-22 14:13:36.400236 | Epoch: 6 | Train Loss: 223.147, Train Time: 4.51 [sec] | Valid Loss: 250.589, Valid Time: 0.91 [sec]
2022-09-22 14:13:41.724833 | Epoch: 7 | Train Loss: 162.804, Train Time: 4.38 [sec] | Valid Loss: 252.941, Valid Time: 0.9

In [44]:
{rs: min(trainer.val_losses) for rs, trainer in zip(random_seeds, trainer_list)}

{32: 104.93993432181222, 1000: 101.17269624982562}

In [36]:
n_epochs = 100

random_seeds = [31, 42, 111, 256, 1024]
trainer_list = []

for random_seed in random_seeds:
    fix_seed(random_seed)
    
    model_name = f'dropout0/EncDec_LSTM4_RS{random_seed}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Random Seed: {random_seed}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = 4
    
    model = EncoderDecoder(lstm_dropout=0)
    trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
    print('='*40, f'(Random Seed = {random_seed}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

======================================== Random Seed: 31 ========================================
2022-09-22 11:18:36.156280 | Epoch: 1 | Train Loss: 876.943, Train Time: 4.28 [sec] | Valid Loss: 839.588, Valid Time: 0.90 [sec]
2022-09-22 11:18:41.512592 | Epoch: 2 | Train Loss: 707.017, Train Time: 4.42 [sec] | Valid Loss: 701.681, Valid Time: 0.91 [sec]
2022-09-22 11:20:19.360117 | Epoch: 20 | Train Loss: 69.023, Train Time: 4.58 [sec] | Valid Loss: 133.947, Valid Time: 0.93 [sec]
2022-09-22 11:20:24.872097 | Epoch: 21 | Train Loss: 67.286, Train Time: 4.58 [sec] | Valid Loss: 177.202, Valid Time: 0.93 [sec]
2022-09-22 11:20:30.510623 | Epoch: 22 | Train Loss: 64.495, Train Time: 4.71 [sec] | Valid Loss: 120.335, Valid Time: 0.93 [sec]
2022-09-22 11:20:35.990742 | Epoch: 23 | Train Loss: 65.872, Train Time: 4.55 [sec] | Valid Loss: 164.204, Valid Time: 0.93 [sec]
2022-09-22 11:20:41.477126 | Epoch: 24 | Train Loss: 66.709, Train Time: 4.56 [sec] | Valid Loss: 152.755, Valid Time: 0.9

In [37]:
{rs: min(trainer.val_losses) for rs, trainer in zip(random_seeds, trainer_list)}

{31: 103.77267637706939,
 42: 103.67929912748791,
 111: 103.62132826305572,
 256: 108.14402716500419,
 1024: 107.05026753743489}

#### dropout

In [28]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/STE-ED-S_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = EncoderDecoder(lstm_dropout=lstm_dropout)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 10:27:15.526256 | Epoch: 1 | Train Loss: 878.726, Train Time: 4.84 [sec] | Valid Loss: 846.147, Valid Time: 0.91 [sec]
2022-09-21 10:27:21.319589 | Epoch: 2 | Train Loss: 711.117, Train Time: 4.85 [sec] | Valid Loss: 702.432, Valid Time: 0.92 [sec]
2022-09-21 10:27:27.307546 | Epoch: 3 | Train Loss: 621.771, Train Time: 5.03 [sec] | Valid Loss: 572.197, Valid Time: 0.92 [sec]
2022-09-21 10:27:33.215185 | Epoch: 4 | Train Loss: 429.419, Train Time: 4.95 [sec] | Valid Loss: 373.649, Valid Time: 0.92 [sec]
2022-09-21 10:27:39.167541 | Epoch: 5 | Train Loss: 305.901, Train Time: 4.99 [sec] | Valid Loss: 278.557, Valid Time: 0.92 [sec]
2022-09-21 10:27:45.123784 | Epoch: 6 | Train Loss: 230.285, Train Time: 4.99 [sec] | Valid Loss: 241.456, Valid Time: 0.92 [sec]
2022-09-21 10:27:51.102090 | Epoch: 7 | Train Loss: 180.855, Train Time: 5.01 [sec] | Valid Loss: 185.272, Valid Time: 0.93 [sec]
20

### 検索無し

In [20]:
TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = 4

In [22]:
n_epochs = 100

model_name = f'dropout0/T-ED_LSTM{TRAFFIC_LSTM_LAYERS}'
log_path = f'./logs/{model_name}.log'
    
model = LSTMOnlyEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-21 09:36:22.298513 | Epoch: 1 | Train Loss: 918.493, Train Time: 3.53 [sec] | Valid Loss: 949.190, Valid Time: 0.66 [sec]
2022-09-21 09:36:26.514432 | Epoch: 2 | Train Loss: 841.664, Train Time: 3.54 [sec] | Valid Loss: 845.555, Valid Time: 0.65 [sec]
2022-09-21 09:36:30.739984 | Epoch: 3 | Train Loss: 743.247, Train Time: 3.55 [sec] | Valid Loss: 754.818, Valid Time: 0.65 [sec]
2022-09-21 09:36:35.121221 | Epoch: 4 | Train Loss: 690.752, Train Time: 3.69 [sec] | Valid Loss: 716.935, Valid Time: 0.66 [sec]
2022-09-21 09:36:39.398556 | Epoch: 5 | Train Loss: 646.277, Train Time: 3.59 [sec] | Valid Loss: 648.486, Valid Time: 0.66 [sec]
2022-09-21 09:36:43.705338 | Epoch: 6 | Train Loss: 553.544, Train Time: 3.61 [sec] | Valid Loss: 540.588, Valid Time: 0.67 [sec]
2022-09-21 09:36:48.040695 | Epoch: 7 | Train Loss: 455.367, Train Time: 3.64 [sec] | Valid Loss: 441.752, Valid Time: 0.67 [sec]
2022-09-21 09:36:52.543385 | Epoch: 8 | Train Loss: 369.151, Train Time: 3.80 [sec] | Vali

In [23]:
n_epochs = 100

model_name = f'dropout0/ST-ED_LSTM{TRAFFIC_LSTM_LAYERS}'
log_path = f'./logs/{model_name}.log'
    
model = CNNLSTMEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-21 09:43:43.013689 | Epoch: 1 | Train Loss: 912.201, Train Time: 3.88 [sec] | Valid Loss: 930.433, Valid Time: 0.67 [sec]
2022-09-21 09:43:47.455132 | Epoch: 2 | Train Loss: 808.645, Train Time: 3.75 [sec] | Valid Loss: 800.028, Valid Time: 0.68 [sec]
2022-09-21 09:43:51.902428 | Epoch: 3 | Train Loss: 708.273, Train Time: 3.75 [sec] | Valid Loss: 726.714, Valid Time: 0.67 [sec]
2022-09-21 09:43:56.360710 | Epoch: 4 | Train Loss: 647.686, Train Time: 3.75 [sec] | Valid Loss: 619.961, Valid Time: 0.68 [sec]
2022-09-21 09:44:00.817166 | Epoch: 5 | Train Loss: 517.256, Train Time: 3.75 [sec] | Valid Loss: 514.511, Valid Time: 0.68 [sec]
2022-09-21 09:44:05.272494 | Epoch: 6 | Train Loss: 387.461, Train Time: 3.76 [sec] | Valid Loss: 390.555, Valid Time: 0.67 [sec]
2022-09-21 09:44:09.865158 | Epoch: 7 | Train Loss: 278.547, Train Time: 3.76 [sec] | Valid Loss: 310.471, Valid Time: 0.81 [sec]
2022-09-21 09:44:14.327069 | Epoch: 8 | Train Loss: 210.173, Train Time: 3.75 [sec] | Vali

In [24]:
n_epochs = 100

model_name = f'dropout0/TE-ED_LSTM{TRAFFIC_LSTM_LAYERS}'
log_path = f'./logs/{model_name}.log'
    
model = LSTMEmbeddingEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-21 09:51:10.304410 | Epoch: 1 | Train Loss: 912.343, Train Time: 3.83 [sec] | Valid Loss: 927.996, Valid Time: 0.80 [sec]
2022-09-21 09:51:14.945596 | Epoch: 2 | Train Loss: 798.887, Train Time: 3.82 [sec] | Valid Loss: 783.008, Valid Time: 0.80 [sec]
2022-09-21 09:51:19.589762 | Epoch: 3 | Train Loss: 694.580, Train Time: 3.82 [sec] | Valid Loss: 713.543, Valid Time: 0.80 [sec]
2022-09-21 09:51:24.367592 | Epoch: 4 | Train Loss: 645.862, Train Time: 3.95 [sec] | Valid Loss: 633.519, Valid Time: 0.80 [sec]
2022-09-21 09:51:29.011748 | Epoch: 5 | Train Loss: 582.415, Train Time: 3.82 [sec] | Valid Loss: 590.764, Valid Time: 0.80 [sec]
2022-09-21 09:51:33.667164 | Epoch: 6 | Train Loss: 454.721, Train Time: 3.83 [sec] | Valid Loss: 404.697, Valid Time: 0.80 [sec]
2022-09-21 09:51:38.309978 | Epoch: 7 | Train Loss: 351.926, Train Time: 3.82 [sec] | Valid Loss: 349.088, Valid Time: 0.80 [sec]
2022-09-21 09:51:43.091945 | Epoch: 8 | Train Loss: 351.234, Train Time: 3.96 [sec] | Vali

In [25]:
n_epochs = 100

model_name = f'dropout0/STE-ED_LSTM{TRAFFIC_LSTM_LAYERS}'
log_path = f'./logs/{model_name}.log'
    
model = EncoderDecoder(lstm_dropout=0, include_search=False)
trainer = train(n_epochs, model, model_name, log_path)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-21 09:58:56.587448 | Epoch: 1 | Train Loss: 912.389, Train Time: 3.89 [sec] | Valid Loss: 931.190, Valid Time: 0.81 [sec]
2022-09-21 09:59:01.282122 | Epoch: 2 | Train Loss: 808.504, Train Time: 3.87 [sec] | Valid Loss: 797.850, Valid Time: 0.81 [sec]
2022-09-21 09:59:06.012310 | Epoch: 3 | Train Loss: 703.964, Train Time: 3.89 [sec] | Valid Loss: 719.027, Valid Time: 0.81 [sec]
2022-09-21 09:59:10.873236 | Epoch: 4 | Train Loss: 617.066, Train Time: 4.03 [sec] | Valid Loss: 586.914, Valid Time: 0.81 [sec]
2022-09-21 09:59:15.609558 | Epoch: 5 | Train Loss: 475.874, Train Time: 3.90 [sec] | Valid Loss: 451.001, Valid Time: 0.81 [sec]
2022-09-21 09:59:20.348169 | Epoch: 6 | Train Loss: 349.717, Train Time: 3.90 [sec] | Valid Loss: 334.077, Valid Time: 0.81 [sec]
2022-09-21 09:59:25.080585 | Epoch: 7 | Train Loss: 263.804, Train Time: 3.90 [sec] | Valid Loss: 253.564, Valid Time: 0.81 [sec]
2022-09-21 09:59:29.947728 | Epoch: 8 | Train Loss: 211.243, Train Time: 4.03 [sec] | Vali

#### random_state

In [45]:
TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = 4
random_seed = 1000

In [46]:
n_epochs = 100

model_name = f'dropout0/T-ED_LSTM4_RS{random_seed}'
log_path = f'./logs/{model_name}.log'
    
model = LSTMOnlyEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-25 08:36:43.111458 | Epoch: 1 | Train Loss: 914.648, Train Time: 3.46 [sec] | Valid Loss: 935.555, Valid Time: 0.66 [sec]
2022-09-25 08:36:47.395831 | Epoch: 2 | Train Loss: 813.627, Train Time: 3.61 [sec] | Valid Loss: 803.783, Valid Time: 0.66 [sec]
2022-09-25 08:36:51.554986 | Epoch: 3 | Train Loss: 709.750, Train Time: 3.48 [sec] | Valid Loss: 727.066, Valid Time: 0.66 [sec]
2022-09-25 08:36:55.734122 | Epoch: 4 | Train Loss: 665.993, Train Time: 3.49 [sec] | Valid Loss: 671.653, Valid Time: 0.66 [sec]
2022-09-25 08:36:59.932528 | Epoch: 5 | Train Loss: 575.780, Train Time: 3.51 [sec] | Valid Loss: 539.896, Valid Time: 0.66 [sec]
2022-09-25 08:37:04.151201 | Epoch: 6 | Train Loss: 442.251, Train Time: 3.53 [sec] | Valid Loss: 423.032, Valid Time: 0.66 [sec]
2022-09-25 08:37:08.378018 | Epoch: 7 | Train Loss: 346.851, Train Time: 3.54 [sec] | Valid Loss: 331.522, Valid Time: 0.66 [sec]
2022-09-25 08:37:12.756303 | Epoch: 8 | Train Loss: 266.694, Train Time: 3.69 [sec] | Vali

In [47]:
n_epochs = 100

model_name = f'dropout0/ST-ED_LSTM4_RS{random_seed}'
log_path = f'./logs/{model_name}.log'
    
model = CNNLSTMEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-25 08:44:00.510737 | Epoch: 1 | Train Loss: 911.838, Train Time: 3.72 [sec] | Valid Loss: 929.950, Valid Time: 0.67 [sec]
2022-09-25 08:44:05.075293 | Epoch: 2 | Train Loss: 807.732, Train Time: 3.87 [sec] | Valid Loss: 798.625, Valid Time: 0.68 [sec]
2022-09-25 08:44:09.518703 | Epoch: 3 | Train Loss: 707.269, Train Time: 3.73 [sec] | Valid Loss: 725.903, Valid Time: 0.68 [sec]
2022-09-25 08:44:13.962886 | Epoch: 4 | Train Loss: 654.070, Train Time: 3.74 [sec] | Valid Loss: 640.062, Valid Time: 0.68 [sec]
2022-09-25 08:44:18.414196 | Epoch: 5 | Train Loss: 535.116, Train Time: 3.75 [sec] | Valid Loss: 516.778, Valid Time: 0.68 [sec]
2022-09-25 08:44:22.998682 | Epoch: 6 | Train Loss: 412.359, Train Time: 3.89 [sec] | Valid Loss: 393.887, Valid Time: 0.68 [sec]
2022-09-25 08:44:27.439095 | Epoch: 7 | Train Loss: 303.933, Train Time: 3.74 [sec] | Valid Loss: 294.893, Valid Time: 0.67 [sec]
2022-09-25 08:44:31.898811 | Epoch: 8 | Train Loss: 245.083, Train Time: 3.75 [sec] | Vali

In [50]:
n_epochs = 100

model_name = f'dropout0/TE-ED_LSTM4_RS{random_seed}'
log_path = f'./logs/{model_name}.log'
    
model = LSTMEmbeddingEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-25 09:36:58.879885 | Epoch: 1 | Train Loss: 914.875, Train Time: 3.76 [sec] | Valid Loss: 938.704, Valid Time: 0.79 [sec]
2022-09-25 09:37:03.322610 | Epoch: 2 | Train Loss: 822.279, Train Time: 3.62 [sec] | Valid Loss: 816.952, Valid Time: 0.79 [sec]
2022-09-25 09:37:07.781082 | Epoch: 3 | Train Loss: 717.647, Train Time: 3.64 [sec] | Valid Loss: 730.051, Valid Time: 0.79 [sec]
2022-09-25 09:37:12.250469 | Epoch: 4 | Train Loss: 667.605, Train Time: 3.65 [sec] | Valid Loss: 680.037, Valid Time: 0.79 [sec]
2022-09-25 09:37:16.743929 | Epoch: 5 | Train Loss: 583.678, Train Time: 3.67 [sec] | Valid Loss: 554.936, Valid Time: 0.79 [sec]
2022-09-25 09:37:21.370787 | Epoch: 6 | Train Loss: 448.154, Train Time: 3.67 [sec] | Valid Loss: 419.955, Valid Time: 0.93 [sec]
2022-09-25 09:37:25.871545 | Epoch: 7 | Train Loss: 342.910, Train Time: 3.68 [sec] | Valid Loss: 329.363, Valid Time: 0.79 [sec]
2022-09-25 09:37:30.388383 | Epoch: 8 | Train Loss: 267.807, Train Time: 3.70 [sec] | Vali

In [49]:
n_epochs = 100

model_name = f'dropout0/STE-ED_LSTM4_RS{random_seed}'
log_path = f'./logs/{model_name}.log'
    
model = EncoderDecoder(lstm_dropout=0, include_search=False)
trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-25 08:59:12.298124 | Epoch: 1 | Train Loss: 902.922, Train Time: 3.89 [sec] | Valid Loss: 908.401, Valid Time: 0.81 [sec]
2022-09-25 08:59:17.009978 | Epoch: 2 | Train Loss: 778.974, Train Time: 3.88 [sec] | Valid Loss: 766.136, Valid Time: 0.81 [sec]
2022-09-25 08:59:21.853420 | Epoch: 3 | Train Loss: 687.037, Train Time: 4.00 [sec] | Valid Loss: 709.010, Valid Time: 0.81 [sec]
2022-09-25 08:59:26.592350 | Epoch: 4 | Train Loss: 604.120, Train Time: 3.90 [sec] | Valid Loss: 567.112, Valid Time: 0.81 [sec]
2022-09-25 08:59:31.331137 | Epoch: 5 | Train Loss: 448.884, Train Time: 3.90 [sec] | Valid Loss: 422.681, Valid Time: 0.81 [sec]
2022-09-25 08:59:36.067184 | Epoch: 6 | Train Loss: 340.401, Train Time: 3.89 [sec] | Valid Loss: 342.251, Valid Time: 0.82 [sec]
2022-09-25 08:59:40.932599 | Epoch: 7 | Train Loss: 280.144, Train Time: 4.03 [sec] | Valid Loss: 268.297, Valid Time: 0.81 [sec]
2022-09-25 08:59:45.667672 | Epoch: 8 | Train Loss: 228.231, Train Time: 3.90 [sec] | Vali

#### dropout

In [29]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/STE-ED_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = EncoderDecoder(lstm_dropout=lstm_dropout, include_search=False)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 12:06:19.729555 | Epoch: 1 | Train Loss: 912.308, Train Time: 4.33 [sec] | Valid Loss: 930.983, Valid Time: 0.95 [sec]
2022-09-21 12:06:24.905756 | Epoch: 2 | Train Loss: 808.207, Train Time: 4.34 [sec] | Valid Loss: 797.684, Valid Time: 0.81 [sec]
2022-09-21 12:06:30.069718 | Epoch: 3 | Train Loss: 704.179, Train Time: 4.33 [sec] | Valid Loss: 719.649, Valid Time: 0.81 [sec]
2022-09-21 12:06:35.240472 | Epoch: 4 | Train Loss: 642.821, Train Time: 4.33 [sec] | Valid Loss: 610.803, Valid Time: 0.81 [sec]
2022-09-21 12:06:40.541625 | Epoch: 5 | Train Loss: 492.357, Train Time: 4.46 [sec] | Valid Loss: 450.967, Valid Time: 0.81 [sec]
2022-09-21 12:06:45.728189 | Epoch: 6 | Train Loss: 344.074, Train Time: 4.34 [sec] | Valid Loss: 324.891, Valid Time: 0.82 [sec]
2022-09-21 12:06:50.909874 | Epoch: 7 | Train Loss: 244.826, Train Time: 4.34 [sec] | Valid Loss: 264.366, Valid Time: 0.81 [sec]
20

In [30]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/T-ED_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = LSTMOnlyEncoderDecoder(lstm_dropout=lstm_dropout)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 13:32:38.742919 | Epoch: 1 | Train Loss: 916.855, Train Time: 4.11 [sec] | Valid Loss: 944.451, Valid Time: 0.68 [sec]
2022-09-21 13:32:43.549089 | Epoch: 2 | Train Loss: 833.289, Train Time: 4.12 [sec] | Valid Loss: 833.622, Valid Time: 0.67 [sec]
2022-09-21 13:32:48.504610 | Epoch: 3 | Train Loss: 733.092, Train Time: 4.25 [sec] | Valid Loss: 745.880, Valid Time: 0.67 [sec]
2022-09-21 13:32:53.328205 | Epoch: 4 | Train Loss: 684.810, Train Time: 4.12 [sec] | Valid Loss: 714.688, Valid Time: 0.67 [sec]
2022-09-21 13:32:58.142882 | Epoch: 5 | Train Loss: 618.394, Train Time: 4.12 [sec] | Valid Loss: 645.846, Valid Time: 0.67 [sec]
2022-09-21 13:33:02.964236 | Epoch: 6 | Train Loss: 510.655, Train Time: 4.12 [sec] | Valid Loss: 504.062, Valid Time: 0.68 [sec]
2022-09-21 13:33:07.770264 | Epoch: 7 | Train Loss: 414.541, Train Time: 4.10 [sec] | Valid Loss: 397.476, Valid Time: 0.68 [sec]
20

In [31]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/ST-ED_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = CNNLSTMEncoderDecoder(lstm_dropout=lstm_dropout)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 14:53:11.428081 | Epoch: 1 | Train Loss: 910.201, Train Time: 4.17 [sec] | Valid Loss: 925.179, Valid Time: 0.68 [sec]
2022-09-21 14:53:16.420540 | Epoch: 2 | Train Loss: 801.009, Train Time: 4.30 [sec] | Valid Loss: 791.218, Valid Time: 0.67 [sec]
2022-09-21 14:53:21.258439 | Epoch: 3 | Train Loss: 702.928, Train Time: 4.13 [sec] | Valid Loss: 723.204, Valid Time: 0.68 [sec]
2022-09-21 14:53:26.187419 | Epoch: 4 | Train Loss: 633.418, Train Time: 4.22 [sec] | Valid Loss: 609.755, Valid Time: 0.68 [sec]
2022-09-21 14:53:31.142880 | Epoch: 5 | Train Loss: 486.849, Train Time: 4.25 [sec] | Valid Loss: 471.773, Valid Time: 0.68 [sec]
2022-09-21 14:53:36.065098 | Epoch: 6 | Train Loss: 348.703, Train Time: 4.21 [sec] | Valid Loss: 353.162, Valid Time: 0.68 [sec]
2022-09-21 14:53:40.916109 | Epoch: 7 | Train Loss: 261.980, Train Time: 4.15 [sec] | Valid Loss: 280.254, Valid Time: 0.67 [sec]
20

In [32]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/TE-ED_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = LSTMEmbeddingEncoderDecoder(lstm_dropout=lstm_dropout)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 16:14:40.505355 | Epoch: 1 | Train Loss: 912.275, Train Time: 4.25 [sec] | Valid Loss: 927.834, Valid Time: 0.81 [sec]
2022-09-21 16:14:45.542849 | Epoch: 2 | Train Loss: 798.748, Train Time: 4.21 [sec] | Valid Loss: 782.714, Valid Time: 0.81 [sec]
2022-09-21 16:14:50.593549 | Epoch: 3 | Train Loss: 694.358, Train Time: 4.22 [sec] | Valid Loss: 712.884, Valid Time: 0.81 [sec]
2022-09-21 16:14:55.752523 | Epoch: 4 | Train Loss: 643.735, Train Time: 4.33 [sec] | Valid Loss: 661.820, Valid Time: 0.80 [sec]
2022-09-21 16:15:00.819255 | Epoch: 5 | Train Loss: 603.858, Train Time: 4.24 [sec] | Valid Loss: 555.274, Valid Time: 0.80 [sec]
2022-09-21 16:15:05.879349 | Epoch: 6 | Train Loss: 478.532, Train Time: 4.23 [sec] | Valid Loss: 616.711, Valid Time: 0.80 [sec]
2022-09-21 16:15:10.903636 | Epoch: 7 | Train Loss: 319.604, Train Time: 4.22 [sec] | Valid Loss: 694.529, Valid Time: 0.81 [sec]
20